In [1]:
using DataFrames, Arrow, UMAP, StatsBase, MLJ

### data loading

In [2]:
celldyn = DataFrame(Arrow.Table("L:/laupodteam/AIOS/Bram/data/CellDyn/celldyn_FULL_transformed_df.feather"));

In [4]:
meas_cols = names(celldyn)[contains.(names(celldyn),r"c_b_|COMBO")];
suspect_cols = names(celldyn)[contains.(names(celldyn),r"^(c_)?s_")];
mode_cols = names(celldyn)[contains.(names(celldyn),"mode")];
alrt_cols = names(celldyn)[contains.(names(celldyn),"Alrt")];
other_c_cols = names(celldyn)[contains.(names(celldyn),r"^c_")];
usual_cols = unique(vcat(meas_cols,suspect_cols,mode_cols,alrt_cols,other_c_cols));

### select independent variables and age

In [5]:
X = celldyn[:,meas_cols]
y = celldyn.age;

### Transform independent variables to normality

In [6]:
Standardizer = @load Standardizer pkg=MLJModels
LR = (@load LassoRegressor pkg=MLJLinearModels verbosity=0)
lr = LR()
XGB = (@load XGBoostRegressor pkg=XGBoost verbosity = 0)
xgb = XGB();


import MLJModels ✔

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\hjoosse3\.julia\packages\MLJModels\K5pPR\src\loading.jl:159


In [7]:
X = dropmissing(X)
y = coerce(collect(skipmissing(y)),autotype(y));

In [8]:
pipe = (X -> coerce(X, autotype(X))) |> Standardizer() |> lr

DeterministicPipeline(
  f = var"#1#2"(), 
  standardizer = Standardizer(
        features = Symbol[], 
        ignore = false, 
        ordered_factor = false, 
        count = false), 
  lasso_regressor = LassoRegressor(
        lambda = 1.0, 
        fit_intercept = true, 
        penalize_intercept = false, 
        scale_penalty_with_samples = true, 
        solver = nothing), 
  cache = true)

In [10]:
range_lr = range(pipe,:(lasso_regressor.lambda);lower =0.001, upper = 10, scale = :log10)
tm_lr = TunedModel(
    model = pipe,
    range = range_lr,
    resampling = CV(nfolds = 10),
    measures=[RootMeanSquaredError(),RSquared()]
)
m = machine(tm_lr,X,y)

Machine trained 0 times; does not cache data
  model: DeterministicTunedModel(model = DeterministicPipeline(f = #1, …), …)
  args: 
    1:	Source @778 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @998 ⏎ `AbstractVector{Count}`


In [12]:
e = evaluate!(m,resampling = CV(nfolds = 5),measures = [rsquared,rmse])